In [1]:
import os, sys
sys.path.append("..")
from config.load_config import load_yaml_config, to_dict, recursive_namespace
import torch
import numpy as np
from torch import Tensor
from importlib import reload

import pytorch_lightning as pl

import models.Model3D as Model3D; Model3D = reload(module=Model3D)
import models.Model4D as Model4D; Model4D = reload(module=Model4D)
import models.lightning.EncoderLightningModule as EncoderLightningModule; EncoderLightningModule = reload(module=EncoderLightningModule)
import models.lightning.DecoderLightningModule as decoder_lightning; decoder_lightning = reload(module=decoder_lightning)

EncoderTemporalSequence = Model4D.EncoderTemporalSequence
DecoderTemporalSequence = Model4D.DecoderTemporalSequence
AutoencoderTemporalSequence = Model4D.AutoencoderTemporalSequence
DecoderStyle = Model4D.DecoderStyle
CineComaEncoder = EncoderLightningModule.CineComaEncoder

TemporalDecoderLightning = decoder_lightning.TemporalDecoderLightning

## Generate dataset

In [2]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule, get_coma_args

config = load_yaml_config("../config_files/config_folded_c_and_s.yaml")
dm = get_datamodule(config)
coma_args = get_coma_args(config, dm)
coma_args["phase_input"] = False

Retrieving synthetic population from cached file.


___

### `EncoderTemporalSequence`

*Status: tested and working*

In [3]:
enc_config = {k: v for k,v in coma_args.items() if k in Model4D.ENCODER_ARGS}
cine_encoder = EncoderTemporalSequence(enc_config, z_aggr_function="DFT", n_timeframes=20)

Let's test the model on a single datapoint:

In [4]:
datapoint = next(iter(dm.train_dataloader()))
s_t = datapoint["s_t"]
s_t.shape

torch.Size([16, 20, 1002, 3])

In [5]:
mu = cine_encoder(s_t)["mu"]
mu.shape

torch.Size([16, 8])

___

### `DecoderStyle`

*Status: tested and working*

In [6]:
dec_c_config = {k: v for k,v in coma_args.items() if k in Model4D.DECODER_C_ARGS}
dec_s_config = {k: v for k,v in coma_args.items() if k in Model4D.DECODER_S_ARGS}

In [7]:
decoder_s = DecoderStyle(dec_s_config, phase_embedding_method="exp")

In [8]:
zc_dim = dec_s_config["latent_dim_content"]
zs_dim = dec_s_config["latent_dim_style"]
zc = Tensor(np.random.random((config.batch_size, zc_dim)))
zs = Tensor(np.random.random((config.batch_size, zs_dim)))

In [9]:
s_hat = decoder_s(zc, zs, coma_args["n_timeframes"])

### `DecoderTemporalSequence`

In [10]:
decoder_4d = DecoderTemporalSequence(dec_c_config, dec_s_config, phase_embedding_method="exp")

___

### `AutoencoderTemporalSequence`

In [11]:
ae_config = [enc_config, dec_c_config, dec_s_config]

In [12]:
autoencoder_4d = AutoencoderTemporalSequence(*ae_config, z_aggr_function="dft", n_timeframes=coma_args["n_timeframes"])

In [13]:
avg_shat, shat_t = autoencoder_4d(s_t)

In [14]:
print(avg_shat.shape)
print(shat_t.shape)

torch.Size([16, 1002, 3])
torch.Size([16, 20, 1002, 3])


___

# PyTorch Lightning modules

Encoder

Decoder

In [16]:
dec_c_config["latent_dim_content"] = dec_s_config["latent_dim_content"] = 9
dec_s_config["latent_dim_style"] = 36

In [17]:
decoder_4d = DecoderTemporalSequence(dec_c_config, dec_s_config, phase_embedding_method="exp")
dec_pl = TemporalDecoderLightning(decoder_4d, config)

In [18]:
trainer = pl.Trainer()
trainer.fit(dec_pl, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name  | Type                    | Params
--------------------------------------------------
0 | model | DecoderTemporalSequence | 207 K 
--------------------------------------------------
207 K     Trainable params
0         Non-trainable params
207 K     Total params
0.831     Total estimated model params size (MB)


A Jupyter Widget

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


A Jupyter Widget

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/closure.py:35: LightningDeprecationWarning: One of the returned values {'recon_loss_s', 'recon_loss_c'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

/home/rodrigo/anaconda3/envs/cardio/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Full autoencoder

In [19]:
# PLEncoder = CineComaEncoder(cine_encoder, config)
# trainer = pl.Trainer()
# trainer.fit(PLEncoder, dm)